In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
    apply_theme,
)
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
df = pd.read_csv(join("..", "data", "Phase2.1", "temporal", "labs", "lab_obs.csv"))
df = df.drop(columns=[df.columns.values.tolist()[0]])

df = df.melt(id_vars=["cohort", "day", "setting", "wave"], var_name="lab")

lab_nice = {
     'alanine.aminotransferase..ALT.': 'alanine aminotransferase (ALT)',
     'albumin': 'albumin',
     'aspartate.aminotransferase..AST.': 'aspartate aminotransferase (AST)',
     'total.bilirubin': 'total bilirubin',
     'C.reactive.protein..CRP...Normal.Sensitivity.': 'C-reactive protein (CRP) (Normal Sensitivity)',
     'creatinine': 'creatinine',
     'Ferritin': 'Ferritin',
     'lactate.dehydrogenase..LDH.': 'lactate dehydrogenase (LDH)',
     'Fibrinogen': 'Fibrinogen',
     'procalcitonin': 'procalcitonin',
     'cardiac.troponin..High.Sensitivity.': 'cardiac troponin (High Sensitivity)',
     'white.blood.cell.count..Leukocytes.': 'white blood cell count (Leukocytes)',
     'lymphocyte.count': 'lymphocyte count',
     'neutrophil.count': 'neutrophil count',
     'D.dimer': 'D-dimer',
     'prothrombin.time..PT.': 'prothrombin time (PT)',
     'cardiac.troponin..Normal.Sensitivity.': 'cardiac troponin (Normal Sensitivity)'
}
df["lab"] = df["lab"].apply(lambda x: lab_nice[x])
df["wave"] = df["wave"].apply(lambda x: x[0].upper() + x[1:])
df["setting"] = df["setting"].apply(lambda x: x[0].upper() + x[1:])

# We are most interested in certain time points
df = df.loc[df["day"].isin([0, 1, 7])]
df.head()

In [ ]:
df["day"].unique()

In [ ]:
def compute_early_late_difference(df):
    ls_df = pd.DataFrame()
    for (setting, lab), lab_setting_df in df.groupby(by=["setting", "lab"]):    
        ls_df = ls_df.append({
            "setting": setting,
            "lab": lab,
            "value_early": lab_setting_df.loc[lab_setting_df["wave"] == "Early"].iloc[0]["value"],
            "value_late": lab_setting_df.loc[lab_setting_df["wave"] == "Late"].iloc[0]["value"]
        }, ignore_index=True)
    df = ls_df
    df["difference"] = df["value_late"] - df["value_early"]
    df["is_left_direction"] = df["difference"] < 0.0
    df["is_right_direction"] = df["difference"] > 0.0
    df = df.sort_values(by="difference")

    return df

In [ ]:
def make_early_late_angle_plot_for_day(day):
    day0_df = df.loc[(df["day"] == day) & (df["cohort"] == "dayX")]
    day0_diff_df = compute_early_late_difference(day0_df)

    lab_sort = day0_diff_df["lab"].unique().tolist()
    
    value_domain = [0.0, 1.0]

    points = alt.Chart().mark_point().encode(
        x=alt.X("wave:O", axis=alt.Axis(title=None)),
        y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients tested"), scale=alt.Scale(domain=value_domain)),
        color=alt.Color("setting:N", legend=alt.Legend(title="Severity"))
    )

    lines = alt.Chart().mark_line().encode(
        x=alt.X("wave:O", axis=alt.Axis(title=None)),
        y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients tested"), scale=alt.Scale(domain=value_domain)),
        color=alt.Color("setting:N", legend=alt.Legend(title="Severity"))
    )
    chart = alt.layer(points, lines, data=day0_df).facet(
        column=alt.Column("lab:N", header=alt.Header(title="Lab", orient="bottom", labelAngle=-60, labelAnchor="middle", labelAlign="right"), sort=lab_sort)
    )

    chart = chart.properties(
        title={
            "text": f"Percent of patients tested, day {day} of wave",
            "subtitle": "Day-specific denominator",
            "subtitleColor": "gray"
        },
    )

    chart = apply_theme(chart)

    return chart

In [ ]:
make_early_late_angle_plot_for_day(0)

In [ ]:
make_early_late_angle_plot_for_day(1)

In [ ]:
make_early_late_angle_plot_for_day(7)

In [ ]:
day0_df = df.loc[(df["day"] == 0) & (df["cohort"] == "dayX")]
day0_diff_df = compute_early_late_difference(day0_df)

lab_sort = day0_diff_df["lab"].unique().tolist()

value_domain = [0.0, 1.0]

points = alt.Chart().mark_point().encode(
    x=alt.X("wave:O", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients"), scale=alt.Scale(domain=value_domain)),
    color=alt.Color("setting:N", legend=alt.Legend(title="Severity")),
    opacity=alt.Opacity("cohort:N", legend=alt.Legend(title="Denominator"))
).properties(height=120)

lines = alt.Chart().mark_line().encode(
    x=alt.X("wave:O", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients"), scale=alt.Scale(domain=value_domain)),
    color=alt.Color("setting:N", legend=alt.Legend(title="Severity")),
    opacity=alt.Opacity("cohort:N", legend=alt.Legend(title="Denominator"))
)
chart = alt.layer(points, lines, data=df).facet(
    column=alt.Column("lab:N", header=alt.Header(title="Lab", orient="bottom", labelAngle=-60, labelAnchor="middle", labelAlign="right"), sort=lab_sort),
    row=alt.Row("day:O", header=alt.Header(title="Day of wave", labelAngle=0, labelFontSize=20))
)

chart = chart.properties(
    title="Percent of patients tested by wave",
)

chart = apply_theme(chart)

chart

In [ ]:
day0_df = df.loc[(df["day"] == 0) & (df["cohort"] == "dayX")]
day0_diff_df = compute_early_late_difference(day0_df)

lab_sort = day0_diff_df["lab"].unique().tolist()

value_domain = [0.0, 1.0]

day_specific_df = df.loc[df["cohort"] == "dayX"]

points = alt.Chart().mark_point().encode(
    x=alt.X("wave:O", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients"), scale=alt.Scale(domain=value_domain)),
    color=alt.Color("setting:N", legend=alt.Legend(title="Severity"))
).properties(height=120)

lines = alt.Chart().mark_line().encode(
    x=alt.X("wave:O", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients"), scale=alt.Scale(domain=value_domain)),
    color=alt.Color("setting:N", legend=alt.Legend(title="Severity"))
)
chart = alt.layer(points, lines, data=day_specific_df).facet(
    column=alt.Column("lab:N", header=alt.Header(title="Lab", orient="bottom", labelAngle=-60, labelAnchor="middle", labelAlign="right"), sort=lab_sort),
    row=alt.Row("day:O", header=alt.Header(title="Day of wave", labelAngle=0, labelFontSize=20))
)

chart = chart.properties(
    title={
            "text": f"Percent of patients tested by wave",
            "subtitle": "Day-specific denominator",
            "subtitleColor": "gray"
        }
)

chart = apply_theme(chart)

chart

In [ ]:
day0_df = df.loc[(df["day"] == 0) & (df["cohort"] == "dayX")]
day0_diff_df = compute_early_late_difference(day0_df)

lab_sort = day0_diff_df["lab"].unique().tolist()

value_domain = [0.0, 1.0]

day_specific_df = df.loc[df["cohort"] == "dayX"]

points = alt.Chart().mark_point().encode(
    x=alt.X("wave:O", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients"), scale=alt.Scale(domain=value_domain)),
    color=alt.Color("setting:N", legend=alt.Legend(title="Severity"))
).properties(height=120)

lines = alt.Chart().mark_line().encode(
    x=alt.X("wave:O", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients"), scale=alt.Scale(domain=value_domain)),
    color=alt.Color("setting:N", legend=alt.Legend(title="Severity"))
)
chart = alt.layer(points, lines, data=day_specific_df).facet(
    column=alt.Column("lab:N", header=alt.Header(title="Lab", orient="bottom", labelAngle=-60, labelAnchor="middle", labelAlign="right"), sort=lab_sort),
    row=alt.Row("day:O", header=alt.Header(title="Day of wave", labelAngle=0, labelFontSize=20))
)

chart = chart.properties(
    title={
            "text": f"Percent of patients tested by wave",
            "subtitle": "Day-specific denominator",
            "subtitleColor": "gray"
        }
)

chart = apply_theme(chart)

chart

In [ ]:
day0_df = df.loc[(df["day"] == 0) & (df["cohort"] == "dayX")]
day0_diff_df = compute_early_late_difference(day0_df)

lab_sort = day0_diff_df["lab"].unique().tolist()

value_domain = [0.0, 1.0]

day_specific_df = df.loc[df["cohort"] == "dayX"]

points = alt.Chart().mark_point().encode(
    x=alt.X("day:O", axis=alt.Axis(title="Day", labelAngle=0, labelAlign="center", labelBaseline="top")),
    y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients"), scale=alt.Scale(domain=value_domain)),
    color=alt.Color("wave:N", legend=alt.Legend(title="Wave"))
).properties(height=120)

lines = alt.Chart().mark_line().encode(
    x=alt.X("day:O", axis=alt.Axis(title="Day", labelAngle=0, labelAlign="center", labelBaseline="top")),
    y=alt.Y("value:Q", axis=alt.Axis(title="Percent of patients"), scale=alt.Scale(domain=value_domain)),
    color=alt.Color("wave:N", legend=alt.Legend(title="Wave"))
)
chart = alt.layer(points, lines, data=day_specific_df).facet(
    column=alt.Column("lab:N", header=alt.Header(title="Lab", orient="bottom", labelAngle=-60, labelAnchor="middle", labelAlign="right"), sort=lab_sort),
    row=alt.Row("setting:N", header=alt.Header(title="Severity"))
)

chart = chart.properties(
    title={
            "text": f"Percent of patients tested by severity and day of wave",
            "subtitle": "Day-specific denominator",
            "subtitleColor": "gray"
        }
)

chart = apply_theme(chart)

chart